<a href="https://colab.research.google.com/github/UniVR-DH/ADHLab/blob/main/lecture02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Testing Wikipedia Python APIs to create a document collection

In [ ]:
# %pip install wikipedia-api

In [ ]:
import gzip
import string
import numpy as np

In [ ]:
import wikipediaapi
wapi_html = wikipediaapi.Wikipedia('MyProjectName (name@studenti.univr.it)',
                              'en',
                              extract_format=wikipediaapi.ExtractFormat.HTML)

wapi_text = wikipediaapi.Wikipedia('MyProjectName (name@studenti.univr.it)',
                                   'en',
                                   extract_format=wikipediaapi.ExtractFormat.WIKI)

In [ ]:
page_py = wapi_text.page('New York City')
print("Page - Exists: %s" % page_py.exists())
#print(page_py.text)
print(langlinks.keys())

Page - Exists: True
dict_keys(['af', 'als', 'am', 'an', 'ang', 'ar', 'arc', 'ary', 'arz', 'as', 'ast', 'awa', 'ay', 'az', 'azb', 'ba', 'ban', 'bar', 'bat-smg', 'bcl', 'be', 'be-x-old', 'bg', 'bh', 'bi', 'bjn', 'bm', 'bn', 'bo', 'br', 'bs', 'bug', 'bxr', 'ca', 'cbk-zam', 'cdo', 'ce', 'ceb', 'ch', 'ckb', 'co', 'crh', 'cs', 'cu', 'cv', 'cy', 'da', 'dag', 'de', 'diq', 'dsb', 'dty', 'ee', 'el', 'eml', 'eo', 'es', 'et', 'eu', 'ext', 'fa', 'fi', 'fiu-vro', 'fj', 'fo', 'fr', 'frp', 'frr', 'fur', 'fy', 'ga', 'gag', 'gan', 'gcr', 'gd', 'gl', 'glk', 'gn', 'got', 'gu', 'gv', 'ha', 'hak', 'he', 'hi', 'hif', 'hr', 'hsb', 'ht', 'hu', 'hy', 'hyw', 'ia', 'id', 'ie', 'ig', 'ik', 'ilo', 'inh', 'io', 'is', 'it', 'ja', 'jam', 'jbo', 'jv', 'ka', 'kaa', 'kab', 'kbp', 'ki', 'kk', 'km', 'kn', 'ko', 'krc', 'ks', 'ku', 'kw', 'ky', 'la', 'lad', 'lb', 'lez', 'lfn', 'li', 'lij', 'lld', 'lmo', 'ln', 'lo', 'lt', 'lv', 'mai', 'map-bms', 'mdf', 'mg', 'mhr', 'mi', 'min', 'mk', 'ml', 'mn', 'mr', 'mrj', 'ms', 'mt', 'mwl',

In [ ]:
train_pages = ['New York City', 'Rome', 'Berlin', 'Buenos Aires', 'Beijing', 'Budapest', 'Canberra', 'Prague', 'Madrid', 'Moscow', 'Vienna', 'Warsaw', 'Oslo', 'Porto']
test_pages = ['Addis Ababa',  'Tom Sawyer', 'Johannes Gutenberg', 'Marcel Proust', 'Google', ' Statoil']
languages = set(['als','en', 'es', 'fo', 'fr', 'da','de', 'hu', 'pl', 'pt', 'ru', 'nl', 'nov'])

In [ ]:
def extract_pairs(_page, languages, remove_punct=False):
  langlinks = _page.langlinks
  language_versions = []

  for k in sorted(langlinks.keys()):
    if k not in languages:
      continue
    v = langlinks[k]
    page_text = v.summary
    if remove_punct:
      page_text = page_text.translate(str.maketrans('', '', string.punctuation))

    language_versions.append((page_text,k))
    # print("%s: %s - %s: %s - %s" % (k, v.language, v.title, v.fullurl, v.summary))
  return language_versions

In [ ]:
training_set = []
for page_name in train_pages:
  page_py = wapi_text.page(page_name)
  training_set.extend(extract_pairs(page_py, languages, True))


In [ ]:
test_set = []
for page_name in test_pages:
  page_py = wapi_text.page(page_name)
  test_set.extend(extract_pairs(page_py, languages, True))


Testing the application of
[Less is More: Parameter-Free Text Classification with Gzip](https://arxiv.org/pdf/2212.09410.pdf)

In [ ]:
training_set=np.array(training_set)
test_set=np.array(test_set)
macro_accuracy = 0
micro_accuracy = { l:0 for l in languages}
count_l = { l:0 for l in languages}
for (x1 , real) in test_set:
   Cx1 = len(gzip.compress(x1.encode ()))
   distance_from_x1 = []
   for (x2 , _) in training_set:
      Cx2 = len(gzip.compress(x2.encode ()))
      x1x2 = " ".join([x1 , x2])
      Cx1x2 = len(gzip.compress(x1x2.encode ()))
      ncd = (Cx1x2 - min(Cx1 , Cx2)) / max( Cx1 , Cx2 )
      distance_from_x1.append( ncd )
   k=5
   sorted_idx = np.argsort(np.array(distance_from_x1))
   top_k_class = list(training_set[sorted_idx[:k], 1])
   predict_class = max(set(top_k_class) , key=top_k_class.count )
   print(real, predict_class, x1[:100] )
   count_l[real]+=1
   if real == predict_class:
    macro_accuracy+=1
    micro_accuracy[real]+=1
print("Macro accuracy", macro_accuracy/len(test_set))
for l in languages:
  if count_l[l] > 0:
    print(l, count_l[l], micro_accuracy[l]/count_l[l])

da da Addis Ababa også Addis Abeba er hovedstad i Etiopien Navnet betyder ny blomst Byen ligger i 2200 til
de als Addis Abeba adˈɪs ˈaːbeba auch aˈbeːba auch Addis Ababa amharisch አዲስ ፡ አበባ adˈdiːs ˈaβəβa  umgangss
es es Adís Abeba1​2​ del amhárico አዲስ አበባ Ädis Äbäba ‘flor nueva’ adaptado en algunos idiomas como Addis A
fo fo Høvuðsstaðurin í Etiopia Addis Abeba amhariskt Āddīs Ābebā er størsti býur í Etiopia Addis Abeba hev
fr als AddisAbeba adis‿abeba  en amharique  አዲስ አበባ Addis Abäba adˈdis ˈabəba  « Nouvelle fleur »  en oromo
hu hu AddiszAbeba Etiópia egyik külön szövetségi államának is számító fővárosa az Afrikai Unió székhelye A
nl nl Addis Abeba Amhaars አዲስ አበባ nieuwe bloem Afaan Oromo Finfinne is de hoofdstad van Ethiopië De stad t
pl pl Addis Abeba amh አዲስ አበባ „Nowy Kwiat” – największe miasto i stolica Etiopii Położone jest w centrum k
pt pt Adis Abeba em amárico አዲስ አበባ AFI adːiːs aβəβa nova flor em oromo Finfinnee é a capital e a maior ci
ru ru Адди́сАбе́ба Finfinnee с ором